## Imports

In [48]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os

In [49]:
df = pd.read_csv('South_Florida_resi_flips_since_start_of_2022 - Sheet1.csv')

## Image URL collector

In [50]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [51]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

df['image_links'] = links

## Article link formatter

In [52]:
len(links)

20

In [53]:
df["short_description"] = df["full_address"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['full_address'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="https://therealdeal.com/miami/2022/04...
1     <a href="https://therealdeal.com/miami/2022/05...
2     <a href="https://therealdeal.com/miami/2022/02...
3     <a href="https://therealdeal.com/miami/2022/07...
4     <a href="https://therealdeal.com/miami/2022/02...
5     <a href="https://therealdeal.com/miami/2022/08...
6     <a href="https://therealdeal.com/miami/2022/03...
7     <a href="https://therealdeal.com/miami/2022/07...
8     <a href="https://therealdeal.com/miami/2022/05...
9     <a href="https://therealdeal.com/miami/2022/06...
10    <a href="https://therealdeal.com/miami/2022/04...
11    <a href="https://therealdeal.com/miami/2022/06...
12    <a href="https://therealdeal.com/miami/2022/06...
13    <a href="https://therealdeal.com/miami/2022/05...
14    <a href="https://therealdeal.com/miami/2022/07...
15    <a href="https://therealdeal.com/miami/2022/01...
16    <a href="https://therealdeal.com/miami/2022/02...
17    <a href="https://therealdeal.com/miami/202

In [54]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/04/05/steve-witkoff-flips-waterfront-miami-beach-home-lot-for-188-jump/" target="_blank" rel="noopener noreferrer">6455 Pine Tree Drive</a> Circle, Miami Beach, FL'

## Google Maps API Geolocater Setup

In [55]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [56]:
df['geo_address'] = df['full_address']
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [57]:
df.columns

Index(['full_address', 'buyer', 'seller', 'old_sale_price', 'old_sale_date',
       'new_sale_price', 'new_sale_date', 'percent_change', 'description',
       'story_link', 'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [58]:
df

,full_address,buyer,seller,old_sale_price,old_sale_date,new_sale_price,new_sale_date,percent_change,description,story_link,image_links,short_description,remaining_desc,description_link,geo_address,loc,point,lat,lon,altitude
0,"6455 Pine Tree Drive Circle, Miami Beach, FL",unknown,Steve Witkoff,"$7,000,000.00",January 2021,"$20,000,000.00",April 2022,185.714286,NaN,https://therealdeal.com/miami/2022/04/05/steve...,https://s13.therealdeal.com/trd/m/up/2022/04/m...,6455 Pine Tree Drive,"Circle, Miami Beach, FL","<a href=""https://therealdeal.com/miami/2022/04...","6455 Pine Tree Drive Circle, Miami Beach, FL","(6455 Pine Tree Dr Cir, Miami Beach, FL 33141,...","(25.8464182, -80.1261529, 0.0)",25.846418,-80.126153,0.0
1,"215 Indian Road, Palm Beach, FL",Sandra and Paul Edgerley,Todd Michael Glaser,"$6,400,000.00",April 2021,"$15,500,000.00",May 2022,142.187500,NaN,https://therealdeal.com/miami/2022/05/27/todd-...,https://s11.therealdeal.com/trd/m/up/2022/05/M...,"215 Indian Road,","Palm Beach, FL","<a href=""https://therealdeal.com/miami/2022/05...","215 Indian Road, Palm Beach, FL","(215 Indian Rd, Palm Beach, FL 33480, USA, (26...","(26.7699831, -80.0398262, 0.0)",26.769983,-80.039826,0.0
2,"6300 North Bay Road, Miami Beach, FL",unknown,Michelle Simkins and Jason Rubell,"$9,700,000.00",January 2021,"$21,000,000.00",January 2022,116.494845,NaN,https://therealdeal.com/miami/2022/02/08/miche...,https://s14.therealdeal.com/trd/m/up/2022/02/m...,"6300 North Bay Road,","Miami Beach, FL","<a href=""https://therealdeal.com/miami/2022/02...","6300 North Bay Road, Miami Beach, FL","(6300 N Bay Rd, Miami Beach, FL 33141, USA, (2...","(25.8437238, -80.12955529999999, 0.0)",25.843724,-80.129555,0.0
3,"12203 Tillinghast Circle, Palm Beach Gardens, FL",unknown,Steven Frankel,"$11,000,000.00",June 2021,"$22,500,000.00",June 2022,104.545455,NaN,https://therealdeal.com/miami/2022/07/15/palm-...,https://s13.therealdeal.com/trd/m/up/2022/07/m...,12203 Tillinghast,"Circle, Palm Beach Gardens, FL","<a href=""https://therealdeal.com/miami/2022/07...","12203 Tillinghast Circle, Palm Beach Gardens, FL","(12203 Tillinghast Cir, Palm Beach Gardens, FL...","(26.8559188, -80.1211601, 0.0)",26.855919,-80.121160,0.0
4,"622 North Flagler Drive, West Palm Beach, FL",William Segel,Pat Carney,"$5,000,000.00",May 2021,"$10,200,000.00",January 2022,104.000000,NaN,https://therealdeal.com/miami/2022/02/04/real-...,https://s12.therealdeal.com/trd/m/up/2022/02/m...,622 North Flagler,"Drive, West Palm Beach, FL","<a href=""https://therealdeal.com/miami/2022/02...","622 North Flagler Drive, West Palm Beach, FL","(622 N Flagler Dr, West Palm Beach, FL 33401, ...","(26.7192802, -80.0496625, 0.0)",26.719280,-80.049662,0.0
5,"14958 Palmwood Road, Palm Beach Gardens, FL",Robert and Myron Miller,Aaron Levine,"$5,882,750.00",July 2021,"$11,000,000.00",July 2022,86.987378,NaN,https://therealdeal.com/miami/2022/08/02/devel...,https://therealdeal.com/miami/wp-content/uploa...,"14958 Palmwood Road,","Palm Beach Gardens, FL","<a href=""https://therealdeal.com/miami/2022/08...","14958 Palmwood Road, Palm Beach Gardens, FL","(14958 Palmwood Rd, Palm Beach Gardens, FL 334...","(26.8964016, -80.0759735, 0.0)",26.896402,-80.075974,0.0
6,"2929 Ocean Trust, Gulf Stream, FL",David J. Miller,Jesse Aron Cohn and Karen Sue Cohn,"$15,900,000.00",May 2021,"$26,700,000.00",March 2022,67.924528,NaN,https://therealdeal.com/miami/2022/03/24/ellio...,https://s11.therealdeal.com/trd/m/up/2022/03/m...,"2929 Ocean Trust,","Gulf Stream, FL","<a href=""https://therealdeal.com/miami/2022/03...","2929 Ocean Trust, Gulf Stream, FL","(Gulf Stream, FL 33483, USA, (26.4936835, -80....","(26.4936835, -80.05504169999999, 0.0)",26.493683,-80.055042,0.0
7,"1371 Royal Palm Way, Boca Raton, FL",Candice and Steven Stark,Jeffrey and Amy Kaplan,"$10,300,000.00",June 2021,"$17,200,000.00",June 2022,66.990291,NaN,https://therealdeal.com/miami/2022/07/07/luxur...,https://s12.therealdeal.com/trd/m/up/2022/07/m...,"1371 Roya

## Correction section

In [59]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

### Change value to percent value
df['percent_change'] = pd.Series(['{0:.2f}%'.format(val) for val in df['percent_change']], index = df.index)

## HTML popup formatter

In [60]:
df.columns

Index(['full_address', 'buyer', 'seller', 'old_sale_price', 'old_sale_date',
       'new_sale_price', 'new_sale_date', 'percent_change', 'description',
       'story_link', 'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [82]:
def popup_html(row):
    i = row
    address = df['description_link'].iloc[i]
    percent_change = df['percent_change'].iloc[i]
    buyer = df['buyer'].iloc[i]
    seller = df['seller'].iloc[i]
    old_sale_price = df['old_sale_price'].iloc[i]
    old_sale_date = df['old_sale_date'].iloc[i]
    new_sale_price = df['new_sale_price'].iloc[i]
    new_sale_date = df['new_sale_date'].iloc[i]
    image = df['image_links'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Buyer: <strong>{}'''.format(buyer) + '''</strong><br>
    Seller: <strong>{}'''.format(seller) + '''</strong><br><br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Increase: <strong>{}'''.format(percent_change) + '''</strong><br><br>
    Previous Price: <strong>{}'''.format(old_sale_price) + '''</strong><br>
    New Price: <strong>{}'''.format(new_sale_price) + '''</strong><br><br>
    Previous Sale Date: <strong>{}'''.format(old_sale_date) + '''</strong><br>
    New Sale Date: <strong>{}'''.format(new_sale_date) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [62]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [63]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/04/05/steve-witkoff-flips-waterfront-miami-beach-home-lot-for-188-jump/" target="_blank" rel="noopener noreferrer">6455 Pine Tree Drive</a> Circle, Miami Beach, FL'

In [83]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=8)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Top South Florida Resi Sale Flips Since January')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [84]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [78]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [79]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/south_florida_top_resi_flips_jan_2022_august_2022
